In [1]:
import os
import random
import numpy as np
import torch
import torchvision
from torchsummary import summary

from fpga_nn_backend.datasets import *
from fpga_nn_backend.training import *
from fpga_nn_backend.evaluation import *
from fpga_nn_backend.models.relu_toy_models import *
from fpga_nn_backend.quantization import *
from fpga_nn_backend.fpga_simple.emulation import *
from fpga_nn_backend.fpga_simple.conversion import *
from fpga_nn_backend.utils import *

In [2]:
print("PyTorch Version:", torch.__version__)
print("Torchvision Version:", torchvision.__version__)
# Detect if we have a GPU available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    print("Using the GPU!")
else:
    print("WARNING: Could not find GPU! Using CPU only")

PyTorch Version: 1.10.0
Torchvision Version: 0.11.1


In [3]:
data_dir = get_rel_pkg_path("dataset/")
weights_dir = get_rel_pkg_path("weights/")
session_dir = get_rel_pkg_path("sessions/")
models_dir = get_rel_pkg_path("models/")

In [4]:
dataset_type = ImageDatasetType.MNIST

In [5]:
orig_datasets = get_img_dataset(data_dir, dataset_type)

In [6]:
datasets = apply_img_transforms(orig_datasets, dataset_type, flatten=True)

In [7]:
dataloaders = get_dataloaders(datasets, 128, 128, num_workers=0)

In [8]:
input_dim = IMG_DATASET_TO_IMG_SIZE_FLAT[dataset_type]
num_classes = IMG_DATASET_TO_NUM_CLASSES[dataset_type]

model = ReLUToyModel(input_dim, num_classes, layer_dims=[])
model = model.to(device)

In [9]:
model = QuantWrapper(model)

In [10]:
summary(model, (input_dim,))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         QuantStub-1                  [-1, 784]               0
            Linear-2                   [-1, 10]           7,840
      ReLUToyModel-3                   [-1, 10]               0
       DeQuantStub-4                   [-1, 10]               0
Total params: 7,840
Trainable params: 7,840
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.01
Params size (MB): 0.03
Estimated Total Size (MB): 0.04
----------------------------------------------------------------


In [11]:
# model.load_state_dict(torch.load(os.path.join(weights_dir, r"Experiment 11-18-2021 11-04-36 PM\Weights Best.pckl")))

In [12]:
# model.load_state_dict(torch.load(os.path.join(weights_dir, r"Experiment 11-20-2021 06-39-40 PM\Weights Best.pckl")))

In [13]:
model.load_state_dict(torch.load(os.path.join(weights_dir, r"Experiment 12-07-2021 11-47-53 PM\Weights Best.pckl")))

<All keys matched successfully>

In [14]:
criterion = get_loss()
criterion = criterion.to(device)

In [15]:
model.eval()
qconfig = torch.quantization.QConfig(
    activation=torch.quantization.MinMaxObserver.with_args(dtype=torch.quint8),
    weight=torch.quantization.MinMaxObserver.with_args(dtype=torch.qint8))
#qconfig = torch.quantization.get_default_qconfig('fbgemm')
model.qconfig = qconfig
#torch.quantization.fuse_modules(model.model.layers, [['0', '1'], ['2', '3'], ['4', '5'], ['6', '7']], inplace=True)
model = torch.quantization.prepare(model)
stats = get_dataloader_stats(dataloaders['test'], model, criterion, device)
model_int8 = torch.quantization.convert(model)

100%|██████████████████████████████████████████████████████████████████████████████████| 79/79 [00:04<00:00, 19.05it/s]


In [16]:
print("Accuracy:", stats['acc'])

Accuracy: 0.9034


In [17]:
model_int8

QuantWrapper(
  (model): ReLUToyModel(
    (layers): Sequential(
      (0): QuantizedLinear(in_features=784, out_features=10, scale=0.10827332735061646, zero_point=125, qscheme=torch.per_tensor_affine)
    )
  )
  (quant): Quantize(scale=tensor([0.0039]), zero_point=tensor([0]), dtype=torch.quint8)
  (dequant): DeQuantize()
)

In [18]:
model_int8.model.layers[0].weight().int_repr()

tensor([[ -8, -23, -24,  ..., -19,  -7,  -9],
        [-18, -23, -17,  ..., -20, -17, -19],
        [ -4, -24, -24,  ..., -14, -15,  -7],
        ...,
        [-19, -24,  -8,  ..., -17,  -7, -22],
        [ -4, -10,  -7,  ..., -16,  -7, -22],
        [-21, -12, -11,  ...,  -7,  -5, -14]], dtype=torch.int8)

In [19]:
model_int8.model.layers[0].weight()

tensor([[ 0.0207, -0.0310, -0.0344,  ..., -0.0172,  0.0241,  0.0172],
        [-0.0138, -0.0310, -0.0103,  ..., -0.0207, -0.0103, -0.0172],
        [ 0.0344, -0.0344, -0.0344,  ...,  0.0000, -0.0034,  0.0241],
        ...,
        [-0.0172, -0.0344,  0.0207,  ..., -0.0103,  0.0241, -0.0276],
        [ 0.0344,  0.0138,  0.0241,  ..., -0.0069,  0.0241, -0.0276],
        [-0.0241,  0.0069,  0.0103,  ...,  0.0241,  0.0310,  0.0000]],
       size=(10, 784), dtype=torch.qint8,
       quantization_scheme=torch.per_tensor_affine, scale=0.003444838570430875,
       zero_point=-14)

In [20]:
stats = get_dataloader_stats(dataloaders['test'], model_int8, criterion, device)

100%|██████████████████████████████████████████████████████████████████████████████████| 79/79 [00:03<00:00, 21.38it/s]


In [21]:
print("Accuracy:", stats['acc'])

Accuracy: 0.9009


In [22]:
def safe_get_param(param):
    if param is not None:
        return param.int_repr().numpy()
    else:
        return None

In [23]:
converted_nn = ConvertedNN((1, 28, 28))

converted_nn.add_flatten_layer((1, 28, 28), 0, 0)

converted_nn.add_dense_layer((784,), (10,), 0, 0,
    weight=safe_get_param(model_int8.model.layers[0].weight()),
    bias=safe_get_param(model_int8.model.layers[0].bias()))

converted_nn.add_output_layer((10,), 0, 0)

In [24]:
model_int8.model.layers[0]

QuantizedLinear(in_features=784, out_features=10, scale=0.10827332735061646, zero_point=125, qscheme=torch.per_tensor_affine)

In [25]:
converted_nn.finalize()

In [26]:
converted_nn.get_layer_info()

{'layers': [{'layer_type': <ConverterLayerType.FLATTEN: 5>,
   'input_shapes': ((1, 28, 28),),
   'output_shape': (784,),
   'output_size': 784,
   'stack_input_indices': (0,),
   'stack_output_index': 0,
   'parameters': None,
   'metadata': None},
  {'layer_type': <ConverterLayerType.DENSE: 0>,
   'input_shapes': ((784,),),
   'output_shape': (10,),
   'output_size': 10,
   'stack_input_indices': (0,),
   'stack_output_index': 0,
   'parameters': {'weight': 0},
   'metadata': {'has_bias': False}},
  {'layer_type': <ConverterLayerType.OUTPUT: 7>,
   'input_shapes': ((10,),),
   'output_shape': (10,),
   'output_size': 10,
   'stack_input_indices': (0,),
   'stack_output_index': 0,
   'parameters': None,
   'metadata': None}]}

In [27]:
with open("test.coe", 'w') as f:
    f.write(converted_nn.generate_parameter_coe())

In [28]:
emulator = FPGAEmulator(converted_nn, bram_reserved_size=303000)

In [29]:
emulator.exec_info

{'input_shape': (1, 28, 28),
 'inital_input_addr': 0,
 'layers': [{'layer_type': <LayerType.DENSE: 0>,
   'config': {'has_bias': None,
    'input_base_addr': 0,
    'weight_base_addr': 0,
    'bias_base_addr': 0,
    'output_base_addr': 784,
    'm_size': 10,
    'chw_size': 784}},
  {'layer_type': <LayerType.MOVE: 5>,
   'config': {'input_base_addr': 784, 'output_base_addr': 0, 'n_size': 10}},
  {'layer_type': <LayerType.OUTPUT: 6>,
   'config': {'output_base_addr': 0, 'n_size': 10}}]}

In [30]:
index = 1
for imgs, labels in dataloaders['test']:
    break
data = (imgs[index] * 255).numpy()
data = (data/2).astype(np.int8)
print(data)

[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0  58  62  85 127 127  75  46   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0  84 126 126 126 126
 126 126 109  15   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0  84 126 126 126 106  71  88 126 126  61   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0  26 125 126 105  16
   6   0   3 103 126  70   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0  38 125 105  12   0   0   0  61 124 126  32   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0  15   9
   0   0   0   0 104 126 126  32   0   0   0   0   

In [31]:
parameters = converted_nn.parameters_info['parameters']
print([p.shape for p in parameters])

def dense_no_bias(w, i):
    pass

def relu(i):
    return np.maximum(0, i)

[(10, 784)]


In [32]:
inputs_per_layer = {}
outputs_per_layer = {}
def get_output(name):
    def hook(model, input, output):
        inputs_per_layer[name] = input
        outputs_per_layer[name] = output.detach()
    return hook

hooks = {}
for name, module in model_int8.named_modules():
    hooks[name] = module.register_forward_hook(get_output(name))

In [33]:
batch = imgs[None, index, :]
model_out = model_int8(batch)

In [34]:
outputs_per_layer.keys()

dict_keys(['quant', 'model.layers.0', 'model.layers', 'model', 'dequant', ''])

In [35]:
inputs_per_layer['model.layers.0'][0].int_repr()

tensor([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0, 116, 125, 171, 255,
         255, 150,  93,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0, 169, 253, 253, 253, 253,
         253, 253, 218,  30,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0, 169, 253, 253, 253, 213, 142,
         176, 253, 253, 122,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,  52, 250

In [36]:
outputs_per_layer['model.layers.0'].int_repr()

tensor([[147, 112, 185, 151,  70, 151, 161,  66, 144,  79]], dtype=torch.uint8)

In [37]:
model_int8.model.layers[0].weight()

tensor([[ 0.0207, -0.0310, -0.0344,  ..., -0.0172,  0.0241,  0.0172],
        [-0.0138, -0.0310, -0.0103,  ..., -0.0207, -0.0103, -0.0172],
        [ 0.0344, -0.0344, -0.0344,  ...,  0.0000, -0.0034,  0.0241],
        ...,
        [-0.0172, -0.0344,  0.0207,  ..., -0.0103,  0.0241, -0.0276],
        [ 0.0344,  0.0138,  0.0241,  ..., -0.0069,  0.0241, -0.0276],
        [-0.0241,  0.0069,  0.0103,  ...,  0.0241,  0.0310,  0.0000]],
       size=(10, 784), dtype=torch.qint8,
       quantization_scheme=torch.per_tensor_affine, scale=0.003444838570430875,
       zero_point=-14)

In [38]:
emulator.execute(data)

{'has_bias': None, 'input_base_addr': 0, 'weight_base_addr': 0, 'bias_base_addr': 0, 'output_base_addr': 784, 'm_size': 10, 'chw_size': 784}
{'input_base_addr': 784, 'output_base_addr': 0, 'n_size': 10}
{'output_base_addr': 0, 'n_size': 10}
Cycles: 33730


[-112528,
 -257851,
 31167,
 -99414,
 -447859,
 -97785,
 -55871,
 -468136,
 -124360,
 -388243]

In [39]:
w = parameters[0]

In [40]:
out = (w.astype(np.int32) @ data.astype(np.int32))

In [41]:
print(out)

[-112528 -257851   31167  -99414 -447859  -97785  -55871 -468136 -124360
 -388243]


In [42]:
np.argsort(outputs_per_layer['model.layers.0'].int_repr().numpy())

array([[7, 4, 9, 1, 8, 0, 3, 5, 6, 2]], dtype=int64)

In [43]:
np.argsort(out)

array([7, 4, 9, 1, 8, 0, 3, 5, 6, 2], dtype=int64)

In [44]:
labels

tensor([7, 2, 1, 0, 4, 1, 4, 9, 5, 9, 0, 6, 9, 0, 1, 5, 9, 7, 3, 4, 9, 6, 6, 5,
        4, 0, 7, 4, 0, 1, 3, 1, 3, 4, 7, 2, 7, 1, 2, 1, 1, 7, 4, 2, 3, 5, 1, 2,
        4, 4, 6, 3, 5, 5, 6, 0, 4, 1, 9, 5, 7, 8, 9, 3, 7, 4, 6, 4, 3, 0, 7, 0,
        2, 9, 1, 7, 3, 2, 9, 7, 7, 6, 2, 7, 8, 4, 7, 3, 6, 1, 3, 6, 9, 3, 1, 4,
        1, 7, 6, 9, 6, 0, 5, 4, 9, 9, 2, 1, 9, 4, 8, 7, 3, 9, 7, 4, 4, 4, 9, 2,
        5, 4, 7, 6, 7, 9, 0, 5])